In [11]:
import os,requests,zlib
import pandas as pd
from py2neo import Graph, NodeMatcher

In [12]:
# Assuming Neo4j has been created with this name
#dbName = "PublicOmics"
dbName = "Testing"
pw = "ngs7"
graph = Graph("bolt://localhost:7687/"+dbName, auth=("neo4j", pw))

In [14]:
data = pd.read_csv('oncokb_biomarker_drug_associations.tsv', sep='\t')
data
##  understanding the levels
#   https://www.oncokb.org/levels
for row in data.itertuples():
    s1 = "MERGE (g:Gene { Symbol: '%s'})"%(row.Gene)
    s2 = "MERGE (m:Mutation { Term: '%s:%s'})"%(row.Gene,row.Alterations)
    s3 = "MERGE (t:TumorType { Name: '%s'})"%(row.TumorType)
    s4 = "MERGE (d:Drug { Name: '%s'})"%(row.Drugs)
    b1 = "MERGE (g)-[:ALTERED_BY]->(m)"
    if row.Level.startswith("R"):
        r1 = "MERGE (m)-[:RESISTANCE{level:'%s'}]->(t)"%(row.Level)
        r2 = "MERGE (m)-[:RESISTANCE{level:'%s'}]->(d)"%(row.Level)
    else:
        r1 = "MERGE (m)-[:ACTIONABILITY{level:'%s'}]->(t)"%(row.Level)
        r2 = "MERGE (m)-[:ACTIONABILITY{level:'%s'}]->(d)"%(row.Level)
    subStmt = "{} {} {} {} {} {} {}".format(s1,s2,s3,s4,r1,r2,b1)
    graph.run(subStmt)

data.head(3)


,Level,Gene,Alterations,TumorType,Drugs
0,3,ERBB2,Oncogenic Mutations,Breast Cancer,Neratinib
1,3,AKT1,E17K,Breast Cancer,AZD5363
2,3,AKT1,E17K,Endometrial Cancer,AZD5363
